In [1]:
import pandas as pd
import requests
from io import StringIO

### Problem-1:

You are given a SQL file link: https://drive.google.com/file/d/1WFt7B84LTHhMueoKmz8W-PRo7xXqmZf3/view?usp=share_link. Read the data by using the file and store it in a excel file. In this data, there are 3 tables named "invoices", "order_leads" and "sales_sql". So create 3 sheets to your excel file.

In [ ]:
# code here
#The SQL file is not found at the given location

# code here
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(host="localhost",user="root",password="",database="temp_db")

# using pandas read the data from a RDBMS table
invoices = pd.read_sql_query('SELECT * FROM invoices',conn)
order_lead = pd.read_sql_query('SELECT * FROM order_leads',conn)
sales_sql = pd.read_sql_query('SELECT * FROM sales_sql',conn)


with pd.ExcelWriter('supermarket.xlsx') as writer:
    invoices.to_excel(writer,sheet_name="invoices")
    order_lead.to_excel(writer,sheet_name="order_lead")
    sales_sql.to_excel(writer,sheet_name="sales_sql")

### Problem-2

Go to the site: https://rapidapi.com/wirefreethought/api/geodb-cities. From here, you have to grab the API and have to choose proper routes to get the cities of different countries. After getting the right API, hit that API and create a dataframe of all the cities that you can get by using the API. Then store the dataframe to a SQL. If you need to create an account or have to subscribe, then do that (it has free subscription but has some limitations. Use that free subscription and modify your accordingly to get all the data).  

In [2]:
### code here
### Rapid API 
import requests

url = "https://wft-geo-db.p.rapidapi.com/v1/geo/adminDivisions"

headers = {
    "X-RapidAPI-Key": "b0f75c828fmsh2b0480a62ab8d23p18178ajsn3267e033ae6a",
    "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

data = response.json()['data']
links = response.json()['links']
metadata = response.json()['metadata']

In [3]:
next_url = links[1]['href']

In [7]:
count = metadata['totalCount']//5
count

12767

In [6]:
import time

num_iters = 0
for i in range(1,count):
    
    num_iters += 1
    
    url = "https://wft-geo-db.p.rapidapi.com"
    page = url + next_url
    
    response = requests.request("GET", page, headers=headers)
    
    print(response.json())
    
    links = response.json()['links']
    
    for item in links:
        if item['rel'] == 'next':
            next_url = item['href']
            
    data.extend(response.json()['data'])
    
    time.sleep(2)
    

{'links': [{'rel': 'first', 'href': '/v1/geo/adminDivisions?offset=0&limit=5'}, {'rel': 'prev', 'href': '/v1/geo/adminDivisions?offset=0&limit=5'}, {'rel': 'next', 'href': '/v1/geo/adminDivisions?offset=10&limit=5'}, {'rel': 'last', 'href': '/v1/geo/adminDivisions?offset=63835&limit=5'}], 'data': [{'id': 3228129, 'wikiDataId': 'Q1650529', 'name': 'Achin', 'country': 'Afghanistan', 'countryCode': 'AF', 'region': 'Nangarhar', 'regionCode': 'NAN', 'regionWdId': 'Q178471', 'latitude': 34.0894, 'longitude': 70.683, 'population': 0}, {'id': 3301438, 'wikiDataId': 'Q2714271', 'name': 'Adraskan District', 'country': 'Afghanistan', 'countryCode': 'AF', 'region': 'Herat Province', 'regionCode': 'HER', 'regionWdId': 'Q182844', 'latitude': 33.705277777, 'longitude': 61.995555555, 'population': 0}, {'id': 3198316, 'wikiDataId': 'Q3918406', 'name': 'Ahmadabad District', 'country': 'Afghanistan', 'countryCode': 'AF', 'region': 'Paktia Zurmat', 'regionCode': 'PIA', 'regionWdId': 'Q182493', 'latitude':

KeyboardInterrupt: 

### Problem 3:

Go to this url: https://www.flipkart.com/search?q=smartphones. This is the url to find phones in flipkart website. You have to extract the below things:
1. image url of the phone
2. name of the image
3. average ratings
4. total ratings
5. total reviews
6. discounted price
7. actual price

Extract all the phones which are available in this website. So you have to use the pagination concept. **Also after requesting every page through the url, please wait for a while (minimum 2-3 seconds), otherwise your IP address can be banned to access the flipkart website later.**

After collecting all the data, save that in a JSON file.

In [8]:
# code here
# code here
import requests
from bs4 import BeautifulSoup
import time

import pandas as pd
import numpy as np

In [9]:
webpage = requests.get('https://www.flipkart.com/search?q=smartphones&page=1').text

In [15]:
webpage

'<!doctype html><html lang="en"><head><link href="https://rukminim2.flixcart.com" rel="preconnect"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/atlas.chunk.8dd48d.css"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.c48a12.css"/><link rel="stylesheet" href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.e4e719.css"/><meta http-equiv="Content-type" content="text/html; charset=utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=Edge"/><meta property="fb:page_id" content="102988293558"/><meta property="fb:admins" content="658873552,624500995,100000233612389"/><link rel="shortcut icon" href="https:///www/promos/new/20150528-140547-favicon-retina.ico"/><link type="application/opensearchdescription+xml" rel="search" href="/osdd.xml?v=2"/><meta property="og:type" content="website"/><meta name="og_site_name" property="og:site_name" content="Flipka

In [10]:
soup = BeautifulSoup(webpage,'lxml')

In [11]:
all_divs = soup.find_all('div', class_="_2kHMtA")

In [14]:
phone_image_url = []
phone_name = []
phone_rating = []
phone_num_ratings = []
phone_price = []

for phone in all_divs:
  try:
    phone_image_url.append(phone.find("img",class_="_396cs4")['src'])
  except:
    phone_image_url.append(np.nan)

  try:
    phone_name.append(phone.find("div",class_="_4rR01T").text.strip())
  except:
    phone_name.append(np.nan)

  try:
    phone_rating.append(phone.find("div",class_="_3LWZlK").text.strip())
  except:
    phone_rating.append(np.nan)

  try:
    phone_num_ratings.append(phone.find_all('span',class_='_2_R_DZ')[0].find_all('span')[0].find_all('span')[0].text.strip())
  except:
    phone_num_ratings.append(np.nan)

  try:
    phone_price.append(phone.find("div",class_="_30jeq3 _1_WHN1").text.strip())
  except:
    phone_price.append(np.nan)


df = pd.DataFrame({'phone_name':phone_name,'phone_price':phone_price,'phone_rating':phone_rating,'phone_num_rating':phone_num_ratings,'phone_img_url':phone_image_url})
df

,phone_name,phone_price,phone_rating,phone_num_rating,phone_img_url
